# Homework Starter — Stage 04: Data Acquisition and Ingestion
Name: 
Date: 

## Objectives
- API ingestion with secrets in `.env`
- Scrape a permitted public table
- Validate and save raw data to `data/raw/`

In [2]:
from dotenv import load_dotenv
import os

load_dotenv()
print('ALPHAVANTAGE_API_KEY loaded?', bool(os.getenv('ALPHAVANTAGE_API_KEY')))

ALPHAVANTAGE_API_KEY loaded? True


In [3]:
import os, pathlib, datetime as dt
import requests
import pandas as pd
from bs4 import BeautifulSoup
from dotenv import load_dotenv

RAW = pathlib.Path('data/raw'); RAW.mkdir(parents=True, exist_ok=True)
load_dotenv(); print('ALPHAVANTAGE_API_KEY loaded?', bool(os.getenv('ALPHAVANTAGE_API_KEY')))

C:\Users\qochi\anaconda3\envs\bootcamp_env\lib\site-packages\requests\__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


ALPHAVANTAGE_API_KEY loaded? True


## Helpers (use or modify)

In [4]:
def ts():
    return dt.datetime.now().strftime('%Y%m%d-%H%M%S')

def save_csv(df: pd.DataFrame, prefix: str, **meta):
    mid = '_'.join([f"{k}-{v}" for k,v in meta.items()])
    path = RAW / f"{prefix}_{mid}_{ts()}.csv"
    df.to_csv(path, index=False)
    print('Saved', path)
    return path

def validate(df: pd.DataFrame, required):
    missing = [c for c in required if c not in df.columns]
    return {'missing': missing, 'shape': df.shape, 'na_total': int(df.isna().sum().sum())}

## Part 1 — API Pull (Required)
Choose an endpoint (e.g., Alpha Vantage or use `yfinance` fallback).

In [7]:
# SYMBOL = 'AAPL'
# USE_ALPHA = bool(os.getenv('ALPHAVANTAGE_API_KEY'))
# if USE_ALPHA:
#     url = 'https://www.alphavantage.co/query'
#     params = {'function':'TIME_SERIES_DAILY_ADJUSTED','symbol':SYMBOL,'outputsize':'compact','apikey':os.getenv('ALPHAVANTAGE_API_KEY')}
#     r = requests.get(url, params=params, timeout=30)
#     r.raise_for_status()
#     js = r.json()
#     key = [k for k in js if 'Time Series' in k][0]
#     df_api = pd.DataFrame(js[key]).T.reset_index().rename(columns={'index':'date','5. adjusted close':'adj_close'})[['date','adj_close']]
#     df_api['date'] = pd.to_datetime(df_api['date']); df_api['adj_close'] = pd.to_numeric(df_api['adj_close'])
# else:
#     import yfinance as yf
#     df_api = yf.download(SYMBOL, period='3mo', interval='1d').reset_index()[['Date','Adj Close']]
#     df_api.columns = ['date','adj_close']

# v_api = validate(df_api, ['date','adj_close']); v_api

In [8]:
SYMBOL = 'AAPL'
df_api = None
USE_ALPHA = bool(os.getenv('ALPHAVANTAGE_API_KEY'))

if USE_ALPHA:
    try:
        url = 'https://www.alphavantage.co/query'
        params = {
            'function':'TIME_SERIES_DAILY',  # free endpoint
            'symbol': SYMBOL,
            'outputsize':'compact',
            'apikey': os.getenv('ALPHAVANTAGE_API_KEY')
        }
        r = requests.get(url, params=params, timeout=30)
        r.raise_for_status()
        js = r.json()
        # Debug: see what we got
        # print(js)
        key_list = [k for k in js if 'Time Series' in k]
        if key_list:
            key = key_list[0]
            df_api = pd.DataFrame(js[key]).T.reset_index().rename(
                columns={'index':'date','4. close':'adj_close'}
            )[['date','adj_close']]
            df_api['date'] = pd.to_datetime(df_api['date'])
            df_api['adj_close'] = pd.to_numeric(df_api['adj_close'])
        else:
            print("Alpha Vantage did not return time series. Using yfinance fallback.")
    except Exception as e:
        print("Alpha Vantage request failed:", e)

# Fallback to yfinance if Alpha Vantage fails
if df_api is None:
    import yfinance as yf
    df_api = yf.download(SYMBOL, period='3mo', interval='1d').reset_index()[['Date','Adj Close']]
    df_api.columns = ['date','adj_close']

# Validate
v_api = validate(df_api, ['date','adj_close'])
print("Validation:", v_api)


Validation: {'missing': [], 'shape': (100, 2), 'na_total': 0}


In [9]:
_ = save_csv(df_api.sort_values('date'), prefix='api', source='alpha' if USE_ALPHA else 'yfinance', symbol=SYMBOL)

Saved data\raw\api_source-alpha_symbol-AAPL_20250820-000752.csv


## Part 2 — Scrape a Public Table (Required)
Replace `SCRAPE_URL` with a permitted page containing a simple table.

In [10]:
#SCRAPE_URL = 'https://example.com/markets-table'  # TODO: replace with permitted page
SCRAPE_URL = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'

headers = {'User-Agent':'AFE-Homework/1.0'}
try:
    resp = requests.get(SCRAPE_URL, headers=headers, timeout=30); resp.raise_for_status()
    soup = BeautifulSoup(resp.text, 'html.parser')
    rows = [[c.get_text(strip=True) for c in tr.find_all(['th','td'])] for tr in soup.find_all('tr')]
    header, *data = [r for r in rows if r]
    df_scrape = pd.DataFrame(data, columns=header)
except Exception as e:
    print('Scrape failed, using inline demo table:', e)
    html = '<table><tr><th>Ticker</th><th>Price</th></tr><tr><td>AAA</td><td>101.2</td></tr></table>'
    soup = BeautifulSoup(html, 'html.parser')
    rows = [[c.get_text(strip=True) for c in tr.find_all(['th','td'])] for tr in soup.find_all('tr')]
    header, *data = [r for r in rows if r]
    df_scrape = pd.DataFrame(data, columns=header)

if 'Price' in df_scrape.columns:
    df_scrape['Price'] = pd.to_numeric(df_scrape['Price'], errors='coerce')
v_scrape = validate(df_scrape, list(df_scrape.columns)); v_scrape

{'missing': [], 'shape': (880, 8), 'na_total': 758}

In [11]:
_ = save_csv(df_scrape, prefix='scrape', site='example', table='markets')

Saved data\raw\scrape_site-example_table-markets_20250820-001155.csv


## Documentation
- API Source: (URL/endpoint/params)
- Scrape Source: (URL/table description)
- Assumptions & risks: (rate limits, selector fragility, schema changes)
- Confirm `.env` is not committed.

**API_SOURCE**


Alpha Vantage (function=TIME_SERIES_DAILY_ADJUSTED, symbol=AAPL)

Fallback: Yahoo Finance yfinance


**SCRAPE_SOURCE**
  
Wikipedia — List of S&P 500 companies (https://en.wikipedia.org/wiki/List_of_S%26P_500_companies)


**Assumptions & risks:**

API may have rate limits

Wikipedia schema may change (columns renamed, etc.)

Internet connection required